# Probando Tensorflow

## MNIST (beginners)

Script para descargar los datos de MNIST

In [1]:
import tensorflow as tf
import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Vemos que los datos se dividen en 2 matrices, una de datos para entrenar (mnist.train) y otra para probar nuestro
aprendizaje (mnist.test).
La matriz train cuenta con 55000 imagenes de 28x28 pixeles, es decir, 55000 vectores de tamano 784 que representan
imagenes de digitos escritos a mano. La matriz `mnist.train.label` es la que indica que digito es cada imagen.

In [3]:
print "Dimensiones de la matriz de imagenes >> ", mnist.train.images.shape
print "Dimension de la matriz de labels >> ", mnist.train.labels.shape
print "Dimensiones de la matriz de prueba >> ", mnist.test.images.shape

Dimensiones de la matriz de imagenes >>  (55000, 784)
Dimension de la matriz de labels >>  (55000, 10)
Dimensiones de la matriz de prueba >>  (10000, 784)


In [4]:
# x es solo un placeholder, es decir no tiene un valor especifico aun
x = tf.placeholder("float", [None, 784])   # La dimension de x, es (lo que sea, 784)
W = tf.Variable(tf.zeros([784,10]))        # Pesos
b = tf.Variable(tf.zeros([10]))            # Bias
y = tf.nn.softmax(tf.matmul(x,W) + b)      # Predicted probability

y_ = tf.placeholder("float", [None,10]) 
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)    # Minimizamos con descenso de
# gradiente, con un grado de aprendizaje de 0.01

# Inicializamos las variables que creamos
init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

# Entrenamos con 1000 iteraciones
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))    # Acertamos ?

accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))   # Con que exactitud ?

print sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})

0.9192


# Deep MINST

In [2]:
# Tensorflow utiliza un backend en C++ para hacer el computa mas eficiente.
# Para establecer una coneccion con este backend usaremos una sesion.
sess = tf.InteractiveSession()


# Nodos para las imagenes de entrada y las clases de salida objetivo
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

# pesos y bias ('tensors' de puros ceros hasta ahora)
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

# inicializamos las variables previas definidas para poder usarlas
# se inician dentro de la sesion que creamos
sess.run(tf.initialize_all_variables())

# ahora implementamos nuestro modelo de regresion (softmax)
y = tf.nn.softmax(tf.matmul(x,W) + b)

# especificamos nuestra funcion costo a minimizar, la cual sera
# 'cross-entropy' entre los objetivos y nuestra prediccion de modelo
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

# usamos descenso de gradiente para minimizar la entropia y repetimos
# esto con un epoc de 1000
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
for i in range(1000):
  batch = mnist.train.next_batch(50)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    
# checamos que tan bien le fue a nuestro modelo comparando
# la etiqueta que nuestro modelo dice es la mas probable para cada entrada
# con la etiqueta correcta.
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

# como 'correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))' nos
# dara una lista de booleanos, convertiremos la lista a 0's y 1's y 
# despues le sacamos la media
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

# ahora evaluamos nuestra precision con los datos de prueba
print accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels})

0.9092
